In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout



df_heart = pd.read_csv('heart.csv')

genHealth = [ 'Poor',  'Fair', 'Good', 'Very good', 'Excellent']
sex = ['Male','Female']
ageCategory =  {'18-24',  '25-29','30-34', '35-39', '40-44', '45-49', '50-54','55-59', '60-64','65-69', '70-74', '75-79',  '80 or older'}
race =  ['White', 'Hispanic', 'Other', 'American Indian/Alaskan Native', 'Black', 'Asian']
diabetic =  ['No', 'No, borderline diabetes',  'Yes',  'Yes (during pregnancy)']
bolean = ['No','Yes']

binary_columns = ['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 
                  'DiffWalking', 'PhysicalActivity', 'Asthma', 
                  'KidneyDisease', 'SkinCancer']

df_heart[binary_columns] = df_heart[binary_columns].apply(lambda x: x.map({'Yes': 1, 'No': 0}))


df_heart['GenHealth'] = pd.Categorical(df_heart['GenHealth'], categories=genHealth, ordered=True)
df_heart['GenHealth'] = df_heart['GenHealth'].cat.codes


df_heart['Sex'] = pd.Categorical(df_heart['Sex'], categories=sex, ordered=True)
df_heart['Sex'] = df_heart['Sex'].cat.codes

df_heart['AgeCategory'] = pd.Categorical(df_heart['AgeCategory'], categories=ageCategory, ordered=True)
df_heart['AgeCategory'] = df_heart['AgeCategory'].cat.codes

df_heart['Race'] = pd.Categorical(df_heart['Race'], categories=race, ordered=True)
df_heart['Race'] = df_heart['Race'].cat.codes

df_heart['Diabetic'] = pd.Categorical(df_heart['Diabetic'], categories=diabetic, ordered=True)
df_heart['Diabetic'] = df_heart['Diabetic'].cat.codes


print(df_heart)

        HeartDisease    BMI  Smoking  ...  Asthma  KidneyDisease  SkinCancer
0                  0  16.60        1  ...       1              0           1
1                  0  20.34        0  ...       0              0           0
2                  0  26.58        1  ...       1              0           0
3                  0  24.21        0  ...       0              0           1
4                  0  23.71        0  ...       0              0           0
...              ...    ...      ...  ...     ...            ...         ...
319790             1  27.41        1  ...       1              0           0
319791             0  29.84        1  ...       1              0           0
319792             0  24.24        0  ...       0              0           0
319793             0  32.81        0  ...       0              0           0
319794             0  46.56        0  ...       0              0           0

[319795 rows x 18 columns]


#### Відбираємо дані для кластеризації та проводимо нормалізацію

In [9]:
X = df_heart.drop('SkinCancer', axis=1)
y = df_heart['SkinCancer']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
print(X_train.shape[1])

model = Sequential([
    Dense(256, activation="relu", input_shape=(X_train.shape[-1], )),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation="relu"),
    BatchNormalization(),
    Dropout(0.5),

    Dense(1, activation="sigmoid"),
    ])
model.summary()

17


/Users/akh/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,705 (284.00 KB)

 Trainable params: 71,681 (280.00 KB)

 Non-trainable params: 1,024 (4.00 KB)

In [11]:

model.compile(
    optimizer='adam', 
    loss="binary_crossentropy", 
    metrics=["accuracy"]
)

model.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    verbose=2,
    epochs=30, 
    batch_size=64
)


Epoch 1/30
3998/3998 - 8s - 2ms/step - accuracy: 0.8970 - loss: 0.3048 - val_accuracy: 0.9056 - val_loss: 0.2740
Epoch 2/30
3998/3998 - 9s - 2ms/step - accuracy: 0.9070 - loss: 0.2762 - val_accuracy: 0.9056 - val_loss: 0.2734
Epoch 3/30
3998/3998 - 9s - 2ms/step - accuracy: 0.9070 - loss: 0.2741 - val_accuracy: 0.9056 - val_loss: 0.2725
Epoch 4/30
3998/3998 - 9s - 2ms/step - accuracy: 0.9070 - loss: 0.2725 - val_accuracy: 0.9056 - val_loss: 0.2714
Epoch 5/30
3998/3998 - 9s - 2ms/step - accuracy: 0.9071 - loss: 0.2718 - val_accuracy: 0.9056 - val_loss: 0.2709
Epoch 6/30
3998/3998 - 10s - 2ms/step - accuracy: 0.9070 - loss: 0.2708 - val_accuracy: 0.9056 - val_loss: 0.2697
Epoch 7/30
3998/3998 - 9s - 2ms/step - accuracy: 0.9071 - loss: 0.2702 - val_accuracy: 0.9056 - val_loss: 0.2688
Epoch 8/30
3998/3998 - 9s - 2ms/step - accuracy: 0.9070 - loss: 0.2694 - val_accuracy: 0.9056 - val_loss: 0.2684
Epoch 9/30
3998/3998 - 9s - 2ms/step - accuracy: 0.9070 - loss: 0.2688 - val_accuracy: 0.9056 -

In [12]:
score = model.evaluate(X_test,y_test, verbose=0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.2628449499607086
Test accuracy: 0.9055801630020142
